In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,3 mins 21 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,3 months and 16 days !!!
H2O cluster name:,h2o_python
H2O cluster total nodes:,2
H2O cluster free memory:,45.38 Gb
H2O cluster total cores:,96
H2O cluster allowed cores:,96
H2O cluster status:,"locked, healthy"


In [3]:
DATAFILE = "/mnt/fs-h2o/Telco-Customer-Churn.csv"
data = h2o.import_file(DATAFILE, destination_frame= "gbm_data")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
data.head()

customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.3,1840.75,No
9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.7,151.65,Yes
9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.1,1949.4,No
6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.8,3046.05,Yes
6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [5]:
data.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [6]:
data['SeniorCitizen'] = data['SeniorCitizen'].asfactor()

In [7]:
# costrusco la lista delle features
features = ['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges']

In [30]:
train, valid, test = data.split_frame([0.7, 0.15], seed=1234, destination_frames= ["gbm_train",
                                                                                         "gbm_valid",
                                                                                         "gbm_test"])

In [31]:
# uso la grid search per ottimizzazione hyperparametri

In [32]:
ntrees_opt = [10, 20, 25, 30, 50, 100, 150, 200, 250]
max_depth_opt = [2,3,4,5]
learn_rate_opt = [0.01, 0.1, 0.5 ]

hyper_parameters = {"ntrees": ntrees_opt, "max_depth": max_depth_opt, "learn_rate": learn_rate_opt}

In [33]:
gs = H2OGridSearch(H2OGradientBoostingEstimator, hyper_params=hyper_parameters)

In [34]:
# è importante dare un id al modello, in modo da poterlo ritrovare facilmente in Flow
%time gs.train(x = features, y = 'Churn', training_frame = train, validation_frame = valid, model_id = "gbm_churn")

gbm Grid Build progress: |████████████████████████████████████████████████| 100%
CPU times: user 4.39 s, sys: 300 ms, total: 4.69 s
Wall time: 6min 29s


In [50]:
grid = gs.get_grid(sort_by="recall", decreasing=False)

In [51]:
grid

       learn_rate max_depth ntrees  \
0             0.5         2     30   
1             0.1         4     20   
2             0.5         3     20   
3             0.5         2    150   
4             0.1         3    200   
..  ..        ...       ...    ...   
103          0.01         3     25   
104          0.01         4     30   
105           0.5         2    250   
106          0.01         4    200   
107           0.5         4     50   

                                                       model_ids recall  
0    Grid_GBM_gbm_train_model_python_1579327511459_6025_model_39    1.0  
1    Grid_GBM_gbm_train_model_python_1579327511459_6025_model_20    1.0  
2    Grid_GBM_gbm_train_model_python_1579327511459_6025_model_18    1.0  
3    Grid_GBM_gbm_train_model_python_1579327511459_6025_model_75    1.0  
4    Grid_GBM_gbm_train_model_python_1579327511459_6025_model_89    1.0  
..                                                           ...    ...  
103  Grid_GBM_gbm_train_m

In [ ]:
# addestro il modello usando i parametri identificati dal grid search (max_depth = 3, num_trees = 25)

In [26]:
train, test = data.split_frame([0.8], seed=1234, destination_frames= ["gbm_train", "gbm_test"])

In [27]:
model = H2OGradientBoostingEstimator(ntrees=25, max_depth=3, learn_rate = 0.1)

In [28]:
# è importante dare un id al modello, in modo da poterlo ritrovare facilmente in Flow
%time model.train(x = features, y = 'Churn', training_frame = train, validation_frame = test, model_id = "gbm_churn")

gbm Model Build progress: |███████████████████████████████████████████████| 100%
CPU times: user 106 ms, sys: 13.4 ms, total: 119 ms
Wall time: 1.53 s


In [29]:
model

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_churn


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,25.0,25.0,3964.0,3.0,3.0,3.0,8.0,8.0,8.0




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.1327710263207317
RMSE: 0.36437758756643046
LogLoss: 0.4134512403959426
Mean Per-Class Error: 0.21912023144691706
AUC: 0.8567744897245274
pr_auc: 0.6748565408395557
Gini: 0.7135489794490548

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.33440962498500937: 


,,No,Yes,Error,Rate
0,No,3399.0,758.0,0.1823,(758.0/4157.0)
1,Yes,407.0,1099.0,0.2703,(407.0/1506.0)
2,Total,3806.0,1857.0,0.2057,(1165.0/5663.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.011594,0.764563,3.564050,3.564050,0.937500,0.772015,0.937500,0.772015,0.041322,0.041322,256.404959,256.404959
1,,2,0.022464,0.744923,3.548209,3.556385,0.933333,0.750762,0.935484,0.761732,0.038567,0.079890,254.820937,255.638496
2,,3,0.031884,0.692060,3.509218,3.542449,0.923077,0.709160,0.931818,0.746199,0.033058,0.112948,250.921805,254.244929
3,,4,0.041304,0.687866,3.216783,3.468175,0.846154,0.688504,0.912281,0.733041,0.030303,0.143251,221.678322,246.817457
4,,5,0.050000,0.665663,3.168044,3.415978,0.833333,0.676746,0.898551,0.723250,0.027548,0.170799,216.804408,241.597796
5,,6,0.100000,0.608673,2.369146,2.892562,0.623188,0.637218,0.760870,0.680234,0.118457,0.289256,136.914601,189.256198
6,,7,0.150725,0.539759,2.335301,2.705022,0.614286,0.574976,0.711538,0.644811,0.118457,0.407713,133.530106,170.502225
7,,8,0.200000,0.479252,2.124453,2.561983,0.558824,0.510824,0.673913,0.611800,0.104683,0.512397,112.445309,156.198347
8,,9,0.300000,0.354571,1.707989,2.277319,0.449275,0.410397,0.599034,0.544665,0.170799,0.683196,70.798898,127.731864
9,,10,0.400000,0.290037,1.074380,1.976584,0.282609,0.320234,0.519928,0.488557,0.107438,0.790634,7.438017,97.658402




Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2020-01-18 06:22:54,0.009 sec,0.0,0.441831,0.579175,0.500000,0.000000,1.000000,0.734063,0.440295,0.576238,0.500000,0.000000,1.000000,0.736957
1,,2020-01-18 06:22:54,0.087 sec,1.0,0.430252,0.553733,0.821648,0.336443,2.618522,0.248278,0.428422,0.550238,0.828374,0.332280,2.670905,0.242029
2,,2020-01-18 06:22:54,0.124 sec,2.0,0.421074,0.534260,0.834740,0.398795,2.767822,0.245806,0.419528,0.531407,0.835916,0.392819,2.733214,0.222464
3,,2020-01-18 06:22:54,0.170 sec,3.0,0.412894,0.517354,0.835546,0.398718,2.767822,0.246512,0.411191,0.514187,0.837113,0.393229,2.733214,0.221739
4,,2020-01-18 06:22:54,0.202 sec,4.0,0.406413,0.503966,0.838904,0.415968,2.813552,0.225322,0.405070,0.501481,0.837552,0.406712,2.757537,0.239130
5,,2020-01-18 06:22:54,0.248 sec,5.0,0.400561,0.491974,0.839485,0.416339,2.813552,0.229560,0.399255,0.489542,0.838179,0.408756,2.757537,0.220290
6,,2020-01-18 06:22:54,0.289 sec,6.0,0.395855,0.482113,0.840236,0.417393,2.813552,0.225146,0.394884,0.480329,0.838527,0.408090,0.000000,0.239130
7,,2020-01-18 06:22:54,0.329 sec,7.0,0.391910,0.473706,0.842956,0.479412,2.864985,0.217023,0.391049,0.472130,0.841156,0.459426,2.860291,0.233333
8,,2020-01-18 06:22:54,0.370 sec,8.0,0.388218,0.465920,0.843872,0.480822,2.864985,0.220554,0.387510,0.464579,0.841233,0.459591,2.860291,0.217391
9,,2020-01-18 06:22:54,0.416 sec,9.0,0.385064,0.459295,0.844832,0.482045,2.864985,0.210313,0.384547,0.458226,0.842121,0.460734,2.860291,0.233333



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Contract,833.265259,1.000000,0.450440
1,tenure,264.621552,0.317572,0.143047
2,InternetService,167.911255,0.201510,0.090768
3,OnlineSecurity,152.558777,0.183085,0.082469
4,TechSupport,109.695290,0.131645,0.059298
5,TotalCharges,105.922485,0.127117,0.057259
6,PaymentMethod,77.992172,0.093598,0.042160
7,MonthlyCharges,68.047768,0.081664,0.036785
8,PaperlessBilling,23.553091,0.028266,0.012732
9,MultipleLines,14.637844,0.017567,0.007913


In [ ]:
model.model_performance(test)

In [52]:
# in questo modoe possiamo scegliere uno dei modelli della griglia
best_gbm = grid.models[0]

In [53]:
best_gbm

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  Grid_GBM_gbm_train_model_python_1579327511459_6025_model_39


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,30.0,30.0,3259.0,2.0,2.0,2.0,4.0,4.0,4.0




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.12775778811336547
RMSE: 0.3574322147112169
LogLoss: 0.39513747161603935
Mean Per-Class Error: 0.21491089246171813
AUC: 0.864016580904567
pr_auc: 0.7000335267189182
Gini: 0.728033161809134

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3400704140815603: 


,,No,Yes,Error,Rate
0,No,2959.0,664.0,0.1833,(664.0/3623.0)
1,Yes,338.0,984.0,0.2557,(338.0/1322.0)
2,Total,3297.0,1648.0,0.2026,(1002.0/4945.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010577,0.870369,3.541028,3.541028,0.909091,0.893787,0.909091,0.893787,0.037453,0.037453,254.102826,254.102826
1,,2,0.020192,0.842973,3.116105,3.338684,0.800000,0.855770,0.857143,0.875683,0.029963,0.067416,211.610487,233.868379
2,,3,0.031731,0.825621,3.245943,3.304960,0.833333,0.835439,0.848485,0.861049,0.037453,0.104869,224.594257,230.495971
3,,4,0.040385,0.802013,3.029546,3.245943,0.777778,0.813673,0.833333,0.850897,0.026217,0.131086,202.954640,224.594257
4,,5,0.051923,0.780271,2.272160,3.029546,0.583333,0.785711,0.777778,0.836411,0.026217,0.157303,127.215980,202.954640
5,,6,0.100000,0.680466,3.116105,3.071161,0.800000,0.716366,0.788462,0.778697,0.149813,0.307116,211.610487,207.116105
6,,7,0.150000,0.596807,1.797753,2.646692,0.461538,0.639475,0.679487,0.732290,0.089888,0.397004,79.775281,164.669164
7,,8,0.200000,0.528490,2.097378,2.509363,0.538462,0.561222,0.644231,0.689523,0.104869,0.501873,109.737828,150.936330
8,,9,0.300000,0.357345,1.610487,2.209738,0.413462,0.438119,0.567308,0.605722,0.161049,0.662921,61.048689,120.973783
9,,10,0.400000,0.259041,1.348315,1.994382,0.346154,0.313713,0.512019,0.532720,0.134831,0.797753,34.831461,99.438202




Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2020-01-18 06:27:36,45.015 sec,0.0,0.442572,0.580596,0.500000,0.000000,1.000000,0.732659,0.436958,0.569899,0.500000,0.000000,1.000000,0.743269
1,,2020-01-18 06:27:36,45.077 sec,1.0,0.405411,0.500875,0.786734,0.172311,2.028096,0.241658,0.399764,0.491130,0.791759,0.171895,2.129338,0.229808
2,,2020-01-18 06:27:36,45.119 sec,2.0,0.386738,0.463549,0.823748,0.373233,2.688120,0.246107,0.380400,0.452057,0.834191,0.390760,2.823077,0.231731
3,,2020-01-18 06:27:36,45.164 sec,3.0,0.378202,0.445262,0.834272,0.558535,0.000000,0.242669,0.374082,0.436770,0.837953,0.547470,0.000000,0.243269
4,,2020-01-18 06:27:36,45.213 sec,4.0,0.374540,0.435798,0.835859,0.558082,3.189054,0.229323,0.369585,0.424894,0.844315,0.553583,0.000000,0.240385
5,,2020-01-18 06:27:36,45.253 sec,5.0,0.371276,0.428146,0.840197,0.564759,3.189054,0.232760,0.367803,0.419257,0.843348,0.552960,3.626501,0.245192
6,,2020-01-18 06:27:36,45.306 sec,6.0,0.369478,0.424100,0.844381,0.659680,3.237614,0.211527,0.366295,0.416236,0.845420,0.647398,3.583521,0.244231
7,,2020-01-18 06:27:36,45.349 sec,7.0,0.367913,0.419397,0.847172,0.662692,3.237614,0.213549,0.365701,0.411698,0.848436,0.642148,3.583521,0.245192
8,,2020-01-18 06:27:36,45.401 sec,8.0,0.366725,0.416723,0.848986,0.667537,3.252648,0.213953,0.365943,0.411149,0.847491,0.641922,3.583521,0.244231
9,,2020-01-18 06:27:37,45.461 sec,9.0,0.365724,0.414161,0.850268,0.669624,3.291679,0.212942,0.365137,0.408905,0.848576,0.647784,3.556424,0.244231



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Contract,164.546829,1.000000,0.369597
1,tenure,75.627876,0.459613,0.169872
2,InternetService,51.208683,0.311210,0.115022
3,OnlineSecurity,40.921719,0.248693,0.091916
4,TotalCharges,36.760780,0.223406,0.082570
5,TechSupport,25.437536,0.154591,0.057137
6,MonthlyCharges,19.142237,0.116333,0.042996
7,PaymentMethod,15.814636,0.096110,0.035522
8,MultipleLines,6.286528,0.038205,0.014120
9,PaperlessBilling,5.210114,0.031663,0.011703
